In [1]:
%load_ext autoreload
%autoreload 1
%aimport redditClient

In [2]:
from redditClient import redditClient
import praw
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [3]:
# subreddit name we interested in getting the hot submissions
subreddit_name = 'financialindependence'

# construct Reddit client
client = redditClient()

# specify which subreddit we are interested in
subreddit = client.subreddit(subreddit_name)

# sanity check, you should see your own username printed out
print(client.user.me())

param_kasana


In [4]:
# Calculate the start and end timestamps for the past month
end_timestamp = int(datetime.now().timestamp())  # Current timestamp
start_timestamp = int((datetime.now() - timedelta(days=30)).timestamp())  # 30 days ago

# Create a DataFrame to store the data
data = {'post_title': [], 'post_body': [], 'comment': [], 'comment_author': [], 'timestamp': []}
df = pd.DataFrame(data)

# Loop through Reddit data within the specified time frame
for submission in subreddit.hot(limit=None):
    if start_timestamp <= submission.created_utc <= end_timestamp:
        post_title = submission.title
        timestamp = pd.to_datetime(submission.created_utc, unit='s')  # Convert to correct timestamp format
        post_body = submission.selftext  # Get the post body
        submission.comments.replace_more(limit=None)  # Retrieve all comments, including 'MoreComments'
        for comment in submission.comments.list():
            if hasattr(comment, 'author') and comment.author:  # Check for 'author' attribute
                comment_text = comment.body
                comment_author = comment.author.name

                # Add data to the DataFrame
                df = df.append({
                    'post_title': post_title,
                    'post_body': post_body,
                    'comment': comment_text,
                    'comment_author': comment_author,
                    'timestamp': timestamp
                }, ignore_index=True)
                

In [5]:
df.to_json('FIRE_raw_data.json', orient='records', lines=True)